In [1]:
!pip install --upgrade pandas

     |████████████████████████████████| 10.0MB 100kB/s eta 0:00:01
  Found existing installation: pandas 0.19.2
    Uninstalling pandas-0.19.2:
      Successfully uninstalled pandas-0.19.2
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pyarrow

     |████████████████████████████████| 48.5MB 36kB/s s eta 0:00:01     |███████████████████████▋        | 35.7MB 28.8MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
from numpy import nan as NA
import os
import gc
from pandas.tseries.offsets import MonthEnd

from azure.storage.blob import BlockBlobService
import pyarrow.parquet as pq
from io import StringIO
from io import BytesIO

In [723]:
YearMonth = '2018-06-01'

In [724]:
#Azure account credentials
azure_storage_account_name = "agedatasciencecocstorage"
azure_storage_account_key = "V/FXG+lIWtYc854wbRA8BxGCLGJijS0NDwLvNHfmAxqRopAL3WHWGIQoHxpArG9lZoX+iY+dvtPYoB379w4MIw=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("You must provide a name and key for an Azure Storage account")

In [725]:
# First, we need to connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)
# There are also methods to list containers and blobs
#containers = blob_service.list_containers()
blobs = blob_service.list_blobs('banking-dm')

In [726]:
# List the blobs in the container.
print("\nList blobs in the container")
#generator = block_blob_service.list_blobs(container_name)
for blob in blobs:
    print("\t Blob name: " + blob.name)


List blobs in the container
	 Blob name: ailiev/Balances_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-06-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-05-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-06-01.parquet.gzip
	 Blob name: ailiev/sample.txt
	 Blob name: data/Customer_economic_sector_lookup.csv
	 Blob name: data/Customer_information_2018-01-01.parquet.gzip
	 Blob name: data/Customer_information_2018-02-01.parquet.gzip
	 Blob name: data/Customer_information_2018-03-01.parquet.gzip
	 Blob name: 

### Read Data

In [727]:
container_name = "banking-dm"
blob_name_parquet = 'data/Monthly_Balance_' + YearMonth + '.parquet.gzip'
blob_name = 'data/Product_lookup.csv'

In [728]:
byte_stream = BytesIO()
try:
   blob_service.get_blob_to_stream(container_name=container_name, blob_name=blob_name_parquet, stream=byte_stream)
   balances = pq.read_table(source=byte_stream).to_pandas()
   # Do work on df ...
finally:
   # Add finally block to ensure closure of the stream
   byte_stream.close()

In [729]:
#balances = pd.read_csv(str(input_path + 'Monthly_updated.csv'), header=0)

In [730]:
balances.head(5)

,YearMonth,Customer_ID,Account_ID,Primary_flag,Activity_flag,Product_code,Credit_limit,Deal_currency,Used_amount,Installment_amount,Credit_auto_PMT_PCT,Credit_card_status,Balance_type,Actual_interest_rate,EOM_balance,AVG_balance
0,2018-05-01,1.0,175610,1,Not Active,PRO02,NaN,EUR,NaN,NaN,NaN,None,Debit,NaN,NaN,NaN
1,2018-05-01,1.0,175610PRO14,1,Not Active,PRO14,NaN,EUR,NaN,NaN,NaN,None,Credit,NaN,NaN,NaN
2,2018-05-01,2.0,32365,1,Active,PRO03,NaN,EUR,NaN,NaN,NaN,None,Debit,NaN,0.00,0.0000
3,2018-05-01,3.0,27354,1,Active,PRO02,NaN,EUR,NaN,NaN,NaN,None,Debit,NaN,14454.53,12575.4411
4,2018-05-01,4.0,121835,1,Active,PRO17,1200.0,EUR,600.0,NaN,0.06,Blocked,Credit,0.27,747.00,903.8700


In [731]:
balances.shape

(282794, 16)

In [732]:
#Data Audit
#Count + Unique Count + Missing values by column; Statistic summary for numeric variables; Frequencies for non-numeric variables
def data_audit(df, excel_name = 'Data_Audit.xlsx'):
    #open excel writer
    writer = pd.ExcelWriter(str(output_path + excel_name))
    #sum up the missing values by columns
    missing = pd.Series(df.isnull().sum(), name='missing')
    #use describe function on all columns, transpose it and join the missings summary from above
    describe = df.describe(include='all').T.join(missing)[['count', 'unique', 'missing']]
    #write to excel
    describe.to_excel(writer, sheet_name="Summary", na_rep='NA')
    #find categorical/numerical columns
    categorical_columns = [col for col in df.columns if df[col].dtype == 'O']
    numerical_columns = [col for col in df.columns if df[col].dtype != 'O']
    #export describe statistics only for numeric variables
    missing_num = pd.Series(df[numerical_columns].isnull().sum(), name='missing')
    df[numerical_columns].describe(include='all').T.join(missing_num).to_excel(writer, sheet_name="Numeric", na_rep='NA')
    #loop all categorical variables, value counts, append Total and export to excel sheet
    for value in categorical_columns:
        dictionary = {}
        dictionary = {'Total': df[value].value_counts(dropna=False).sum()}
        df[value].value_counts(dropna=False).append(pd.Series(dictionary)).to_excel(writer, sheet_name=value, na_rep='NA')
    #close excel writer
    writer.save()
    del missing, describe, missing_num
    gc.collect()

In [733]:
#data_audit(balances, 'Balances_Data_Audit.xlsx')

In [734]:
blobstring = blob_service.get_blob_to_text(container_name, blob_name).content
product_codes = pd.read_csv(StringIO(blobstring), header=0, sep=';')

In [735]:
#product_codes

In [736]:
product_codes.fillna('NA', inplace=True)

In [737]:
dic = {'Saving Accounts':'SA', 'Current Accounts':'CA', 'Time Deposits':'TD', 'Short Term':'ST',
       'Long Term':'LT', '1 month':'', '3 months':'', '6 months':'', '12 months':'', '24 months':'',
       'Mortgage':'MG', 'Personal Loans':'PL', 'Fixed':'F', 'Variable':'V', 'Secured':'SEC',
       'Non-secured':'NON_SEC', 'Overdraft':'OVD', 'Auto Loans':'AL', 'Credit Cards':'CC', 'NA':''}

In [738]:
def reclass(df, dic):
    #arr = []
    df_cols = df.columns
    df_cols = [x for x in df_cols if x not in ['Code']]
    df_new = pd.DataFrame(df['Code'], index=df.index)
    for col in df_cols:
        arr = []
        arr.append([dic.get(n, n) for n in df[col].values])
        df_new[col]=arr[0]
    return df_new[df_new.columns[:-1]]

In [739]:
product_codes_reclass = reclass(product_codes, dic)

In [740]:
product_codes_reclass['PROD'] = product_codes_reclass[['Level 1', 'Level 2', 'Level 3']].apply(lambda x: '_'.join(x), axis=1).str.upper()

In [741]:
product_codes_reclass['PROD'] = product_codes_reclass['PROD'].str.rstrip('_ __')

In [742]:
product_codes_reclass = product_codes_reclass[['Code', 'Level 1','PROD']]

In [743]:
product_codes_reclass.columns=['Code', 'CAT','PROD']

In [744]:
product_codes_reclass

,Code,CAT,PROD
0,PRO01,SA,SA
1,PRO02,CA,CA_STANDARD
2,PRO03,CA,CA_PAYROLL
3,PRO04,TD,TD_ST
4,PRO05,TD,TD_ST
5,PRO06,TD,TD_ST
6,PRO07,TD,TD_LT
7,PRO08,TD,TD_LT
8,PRO09,MG,MG_NEW HOUSE
9,PRO10,MG,MG_RENOVATION


In [745]:
# replace spaces and NA
#product_codes['Product'] = product_codes['Product'].str.replace(' NA', '', regex=True)

In [746]:
# replace spaces
#product_codes['Product'] = product_codes['Product'].str.replace(' ', '_', regex=True)

In [747]:
bal_prod_codes = balances.merge(product_codes_reclass, how='left', left_on='Product_code', right_on='Code')

In [748]:
del bal_prod_codes['Product_code']
del bal_prod_codes['Code']

In [749]:
bal_prod_codes.rename(columns={'Installment_amount':'INSA', 'Credit_limit':'CLT', 'AVG_balance':'BAL'}, inplace=True)

In [750]:
bal_prod_codes.head()

,YearMonth,Customer_ID,Account_ID,Primary_flag,Activity_flag,CLT,Deal_currency,Used_amount,INSA,Credit_auto_PMT_PCT,Credit_card_status,Balance_type,Actual_interest_rate,EOM_balance,BAL,CAT,PROD
0,2018-05-01,1.0,175610,1,Not Active,NaN,EUR,NaN,NaN,NaN,None,Debit,NaN,NaN,NaN,CA,CA_STANDARD
1,2018-05-01,1.0,175610PRO14,1,Not Active,NaN,EUR,NaN,NaN,NaN,None,Credit,NaN,NaN,NaN,OVD,OVD
2,2018-05-01,2.0,32365,1,Active,NaN,EUR,NaN,NaN,NaN,None,Debit,NaN,0.00,0.0000,CA,CA_PAYROLL
3,2018-05-01,3.0,27354,1,Active,NaN,EUR,NaN,NaN,NaN,None,Debit,NaN,14454.53,12575.4411,CA,CA_STANDARD
4,2018-05-01,4.0,121835,1,Active,1200.0,EUR,600.0,NaN,0.06,Blocked,Credit,0.27,747.00,903.8700,CC,CC_VISA_ELECTRON


In [751]:
blob_name_parquet = 'data/Ownership_' + YearMonth + '.parquet.gzip'

In [752]:
byte_stream = BytesIO()
try:
   blob_service.get_blob_to_stream(container_name=container_name, blob_name=blob_name_parquet, stream=byte_stream)
   ownership = pq.read_table(source=byte_stream).to_pandas()
   # Do work on df ...
finally:
   # Add finally block to ensure closure of the stream
   byte_stream.close()

In [753]:
ownership.head()

,YearMonth,Customer_ID,Account_ID,Product_code,Opening_date,Closing_date,Expiry_date,Debit_card,Debit_card_opening_date,Debit_card_expiry_date,Credit_card_status,Deal_currency,Deal_term_in_months,Loan_interest_type,Loan_restructure_code
0,2018-05-01,1.0,175610,PRO02,2013-09-19,None,None,False,None,None,None,EUR,NaN,None,NaN
1,2018-05-01,1.0,175610PRO14,PRO14,2018-01-15,None,None,None,None,None,None,EUR,NaN,None,NaN
2,2018-05-01,2.0,32365,PRO03,2000-04-10,None,None,True,2013-04-19,2019-04-19,None,EUR,NaN,None,NaN
3,2018-05-01,3.0,27354,PRO02,1999-02-06,None,None,False,None,None,None,EUR,NaN,None,NaN
4,2018-05-01,4.0,121835,PRO17,2010-07-06,None,2019-07-06,False,None,None,Blocked,EUR,36.0,Var,NaN


In [754]:
ownership.shape

(282794, 15)

In [755]:
#data_audit(ownership, 'Ownership_Data_Audit.xlsx')

In [756]:
ownership['YearMonth'] = pd.to_datetime(ownership['YearMonth'], format="%Y-%m-%d") + MonthEnd(1)
ownership['Opening_date'] = pd.to_datetime(ownership['Opening_date'])
ownership['Closing_date'] = pd.to_datetime(ownership['Closing_date'])
ownership['Expiry_date'] = pd.to_datetime(ownership['Expiry_date'])
ownership['Debit_card_opening_date'] = pd.to_datetime(ownership['Debit_card_opening_date'])

In [757]:
ownership['Mature_Months'] = np.where(pd.notnull(ownership['Opening_date']), 
                         (ownership['YearMonth'] - ownership['Opening_date'])/np.timedelta64(1,'M'), -1)
ownership['Closed_Months'] = np.where(pd.notnull(ownership['Closing_date']), 
                         (ownership['YearMonth'] - ownership['Closing_date'])/np.timedelta64(1,'M'), -1)
ownership['Months_Till_Exp'] = np.where(pd.notnull(ownership['Expiry_date']), 
                         (ownership['Expiry_date'] - ownership['YearMonth'])/np.timedelta64(1,'M'), -1)
ownership['DC_Mature_Months'] = np.where(pd.notnull(ownership['Debit_card_opening_date']), 
                         (ownership['YearMonth'] - ownership['Debit_card_opening_date'])/np.timedelta64(1,'M'), -1)

In [758]:
ownership['Mature_Months'] = ownership['Mature_Months'].astype('int')
ownership['Closed_Months'] = ownership['Closed_Months'].astype('int')
ownership['Months_Till_Exp'] = ownership['Months_Till_Exp'].astype('int')
ownership['DC_Mature_Months'] = ownership['DC_Mature_Months'].astype('int')

In [759]:
drop_cols = ['Opening_date', 'Closing_date', 'Expiry_date', 'Debit_card_opening_date', 'Debit_card_expiry_date', 
            'Deal_currency', 'Loan_interest_type', 'YearMonth', 'Product_code', 'Credit_card_status']

In [760]:
own_cols = [var for var in ownership.columns if var not in drop_cols]

In [761]:
own_cols

['Customer_ID',
 'Account_ID',
 'Debit_card',
 'Deal_term_in_months',
 'Loan_restructure_code',
 'Mature_Months',
 'Closed_Months',
 'Months_Till_Exp',
 'DC_Mature_Months']

In [762]:
ownership=ownership[own_cols]

In [763]:
ownership.head()

,Customer_ID,Account_ID,Debit_card,Deal_term_in_months,Loan_restructure_code,Mature_Months,Closed_Months,Months_Till_Exp,DC_Mature_Months
0,1.0,175610,False,NaN,NaN,56,-1,-1,-1
1,1.0,175610PRO14,None,NaN,NaN,4,-1,-1,-1
2,2.0,32365,True,NaN,NaN,217,-1,-1,61
3,3.0,27354,False,NaN,NaN,231,-1,-1,-1
4,4.0,121835,False,36.0,NaN,94,-1,13,-1


In [764]:
bal_own = bal_prod_codes.merge(ownership, how='left', left_on=['Account_ID', 'Customer_ID'],
                      right_on=['Account_ID', 'Customer_ID'])

In [765]:
bal_own.head()

,YearMonth,Customer_ID,Account_ID,Primary_flag,Activity_flag,CLT,Deal_currency,Used_amount,INSA,Credit_auto_PMT_PCT,...,BAL,CAT,PROD,Debit_card,Deal_term_in_months,Loan_restructure_code,Mature_Months,Closed_Months,Months_Till_Exp,DC_Mature_Months
0,2018-05-01,1.0,175610,1,Not Active,NaN,EUR,NaN,NaN,NaN,...,NaN,CA,CA_STANDARD,False,NaN,NaN,56,-1,-1,-1
1,2018-05-01,1.0,175610PRO14,1,Not Active,NaN,EUR,NaN,NaN,NaN,...,NaN,OVD,OVD,None,NaN,NaN,4,-1,-1,-1
2,2018-05-01,2.0,32365,1,Active,NaN,EUR,NaN,NaN,NaN,...,0.0000,CA,CA_PAYROLL,True,NaN,NaN,217,-1,-1,61
3,2018-05-01,3.0,27354,1,Active,NaN,EUR,NaN,NaN,NaN,...,12575.4411,CA,CA_STANDARD,False,NaN,NaN,231,-1,-1,-1
4,2018-05-01,4.0,121835,1,Active,1200.0,EUR,600.0,NaN,0.06,...,903.8700,CC,CC_VISA_ELECTRON,False,36.0,NaN,94,-1,13,-1


In [766]:
bal_own.shape

(282798, 24)

In [767]:
del balances, bal_prod_codes, ownership, product_codes
gc.collect()

21

Aggregations

In [768]:
def aggregated_index(df, dic):
    arr = []
    for val in df.columns.values:
        arr.append([dic.get(n, n) for n in val])
    columns = pd.Index("_".join(i) for i in arr)
    columns = [x.upper() for x in columns]
    return columns

In [769]:
aggreg_prod = pd.pivot_table(bal_own, values=['BAL', 'Account_ID',], index=['Customer_ID'], columns=['PROD'], 
                             aggfunc={'BAL': [np.mean], 'Account_ID': [len]})

In [770]:
dic_prod = {'mean':'AVG', 'len':'CNT', 'sum':'SUM', 'Account_ID':''}

In [771]:
col_agg_prod = aggregated_index(aggreg_prod, dic_prod)
col_agg_prod = [item.strip('_') for item in col_agg_prod]
col_agg_prod = [item.replace(' ', '_') for item in col_agg_prod]
aggreg_prod.columns = col_agg_prod
aggreg_prod.reset_index(inplace=True)

In [772]:
aggreg_prod.head()

,Customer_ID,CNT_AL_PROMOTION,CNT_AL_STANDARD,CNT_CA_PAYROLL,CNT_CA_STANDARD,CNT_CC_MASTERCARD_GOLD,CNT_CC_MASTERCARD_PLATINUM,CNT_CC_MASTERCARD_STANDARD,CNT_CC_VISA_ELECTRON,CNT_CC_VISA_GOLD,...,BAL_AVG_CC_VISA_PLATINUM,BAL_AVG_MG_NEW_HOUSE,BAL_AVG_MG_RENOVATION,BAL_AVG_OVD,BAL_AVG_PL_F_NON_SEC,BAL_AVG_PL_F_SEC,BAL_AVG_PL_V,BAL_AVG_SA,BAL_AVG_TD_LT,BAL_AVG_TD_ST
0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [773]:
aggreg_prod_flag = pd.pivot_table(bal_own, values=['Account_ID'], index=['Customer_ID'], columns=['PROD'], 
                             aggfunc={'Account_ID': [len]})

In [774]:
dic_prod_flag = {'mean':'AVG', 'len':'FLAG', 'sum':'SUM', 'Account_ID':''}

In [775]:
col_agg_prod_flag = aggregated_index(aggreg_prod_flag, dic_prod_flag)
col_agg_prod_flag = [item.strip('_') for item in col_agg_prod_flag]
col_agg_prod_flag = [item.replace(' ', '_') for item in col_agg_prod_flag]
aggreg_prod_flag.columns = col_agg_prod_flag


In [776]:
aggreg_prod_flag.fillna(0,inplace=True)

In [777]:
aggreg_prod_flag.where(aggreg_prod_flag<=1, 1, inplace=True)

In [778]:
aggreg_prod_flag.reset_index(inplace=True)

In [779]:
aggreg_prod_flag.head()

,Customer_ID,FLAG_AL_PROMOTION,FLAG_AL_STANDARD,FLAG_CA_PAYROLL,FLAG_CA_STANDARD,FLAG_CC_MASTERCARD_GOLD,FLAG_CC_MASTERCARD_PLATINUM,FLAG_CC_MASTERCARD_STANDARD,FLAG_CC_VISA_ELECTRON,FLAG_CC_VISA_GOLD,FLAG_CC_VISA_PLATINUM,FLAG_MG_NEW_HOUSE,FLAG_MG_RENOVATION,FLAG_OVD,FLAG_PL_F_NON_SEC,FLAG_PL_F_SEC,FLAG_PL_V,FLAG_SA,FLAG_TD_LT,FLAG_TD_ST
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [780]:
bal_own_credit = bal_own[bal_own['CAT'].isin(['CC', 'AL', 'MG', 'PL', 'OVD'])]

In [781]:
bal_own_credit.shape

(122177, 24)

In [782]:
aggreg_prod_credit = pd.pivot_table(bal_own_credit, values=['CLT', 'INSA'], index=['Customer_ID'], columns=['PROD'], 
                             aggfunc={'CLT':[sum], 'INSA':[sum]})

In [783]:
dic_prod_credit = {'mean':'AVG', 'len':'CNT', 'sum':''}

In [784]:
col_agg_prod_credit = aggregated_index(aggreg_prod_credit, dic_prod_credit)
col_agg_prod_credit = [item.strip('_') for item in col_agg_prod_credit]
col_agg_prod_credit = [item.replace('__', '_') for item in col_agg_prod_credit]
col_agg_prod_credit = [item.replace(' ', '_') for item in col_agg_prod_credit]
aggreg_prod_credit.columns = col_agg_prod_credit
aggreg_prod_credit.reset_index(inplace=True)

In [785]:
aggreg_prod_credit.head()

,Customer_ID,CLT_AL_PROMOTION,CLT_AL_STANDARD,CLT_CC_MASTERCARD_GOLD,CLT_CC_MASTERCARD_PLATINUM,CLT_CC_MASTERCARD_STANDARD,CLT_CC_VISA_ELECTRON,CLT_CC_VISA_GOLD,CLT_CC_VISA_PLATINUM,CLT_MG_NEW_HOUSE,...,INSA_CC_MASTERCARD_STANDARD,INSA_CC_VISA_ELECTRON,INSA_CC_VISA_GOLD,INSA_CC_VISA_PLATINUM,INSA_MG_NEW_HOUSE,INSA_MG_RENOVATION,INSA_OVD,INSA_PL_F_NON_SEC,INSA_PL_F_SEC,INSA_PL_V
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,4.0,NaN,NaN,NaN,7800.0,NaN,1200.0,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,5.0,NaN,NaN,NaN,NaN,NaN,1200.0,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.0,NaN,NaN,NaN,6400.0,NaN,NaN,2800.0,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [786]:
aggreg_cat_credit = pd.pivot_table(bal_own_credit, values=['CLT'], index=['Customer_ID'], columns=['CAT'], 
                                  aggfunc={'CLT':[sum]})

In [787]:
dic_cat_credit = {'mean':'AVG', 'len':'CNT', 'sum':''}

In [788]:
col_agg_cat_credit = aggregated_index(aggreg_cat_credit, dic_cat_credit)
col_agg_cat_credit = [item.strip('_') for item in col_agg_cat_credit]
col_agg_cat_credit = [item.replace('__', '_') for item in col_agg_cat_credit]
col_agg_cat_credit = [item.replace(' ', '_') for item in col_agg_cat_credit]
col_agg_cat_credit = ['CAT_'+item for item in col_agg_cat_credit]
aggreg_cat_credit.columns = col_agg_cat_credit
aggreg_cat_credit.reset_index(inplace=True)

In [789]:
aggreg_cat_credit.head()

,Customer_ID,CAT_CLT_AL,CAT_CLT_CC,CAT_CLT_MG,CAT_CLT_OVD,CAT_CLT_PL
0,1.0,NaN,NaN,NaN,0.0,NaN
1,4.0,NaN,9000.0,NaN,0.0,NaN
2,5.0,NaN,1200.0,NaN,NaN,NaN
3,6.0,NaN,9200.0,NaN,NaN,NaN
4,7.0,NaN,NaN,NaN,0.0,NaN


In [790]:
aggreg_cat = pd.pivot_table(bal_own, values=['Account_ID', 'BAL'], index=['Customer_ID'], columns=['CAT'], 
                             aggfunc={'Account_ID': [len], 'BAL': [sum]})

In [791]:
dic_cat = {'mean':'AVG', 'len':'CNT', 'sum':'', 'Account_ID':''}

In [792]:
col_agg_cat = aggregated_index(aggreg_cat, dic_cat)
col_agg_cat = [item.strip('_') for item in col_agg_cat]
col_agg_cat = [item.replace('__', '_') for item in col_agg_cat]
col_agg_cat = [item.replace(' ', '_') for item in col_agg_cat]
col_agg_cat = ['CAT_'+item for item in col_agg_cat]
aggreg_cat.columns = col_agg_cat
aggreg_cat.reset_index(inplace=True)

In [793]:
aggreg_cat.head()

,Customer_ID,CAT_CNT_AL,CAT_CNT_CA,CAT_CNT_CC,CAT_CNT_MG,CAT_CNT_OVD,CAT_CNT_PL,CAT_CNT_SA,CAT_CNT_TD,CAT_BAL_AL,CAT_BAL_CA,CAT_BAL_CC,CAT_BAL_MG,CAT_BAL_OVD,CAT_BAL_PL,CAT_BAL_SA,CAT_BAL_TD
0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,0.0,NaN,NaN,NaN
1,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12575.4411,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,1.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,0.0000,903.87,NaN,0.0,NaN,NaN,NaN
4,5.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,847.80,NaN,NaN,NaN,NaN,NaN


In [794]:
#aggreg_cat.iloc[:5, :9]

In [795]:
aggreg_cat_flag = aggreg_cat.iloc[:, :9].copy(deep=True)

In [796]:
aggreg_cat_flag.set_index('Customer_ID', inplace=True)

In [797]:
aggreg_cat_flag.fillna(0,inplace=True)

In [798]:
aggreg_cat_flag.where(aggreg_cat_flag<=1, 1, inplace=True)

In [799]:
aggreg_cat_flag.head()

,CAT_CNT_AL,CAT_CNT_CA,CAT_CNT_CC,CAT_CNT_MG,CAT_CNT_OVD,CAT_CNT_PL,CAT_CNT_SA,CAT_CNT_TD
Customer_ID,,,,,,,,
1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
5.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [800]:
#aggreg_cat_flag.where(aggreg_cat_flag.notnull(), -5, inplace=True)

In [801]:
#aggreg_cat_flag.where(aggreg_cat_flag<=1, 1, inplace=True)

In [802]:
#aggreg_cat_flag.where(aggreg_cat_flag!=-5, np.NaN, inplace=True)

In [803]:
aggreg_cat_flag_cols = [x.replace('CNT', 'FLAG') for x in aggreg_cat_flag.columns]

In [804]:
aggreg_cat_flag.columns = aggreg_cat_flag_cols

In [805]:
aggreg_cat_flag.reset_index(inplace=True)

In [806]:
aggreg_cat_flag.head()

,Customer_ID,CAT_FLAG_AL,CAT_FLAG_CA,CAT_FLAG_CC,CAT_FLAG_MG,CAT_FLAG_OVD,CAT_FLAG_PL,CAT_FLAG_SA,CAT_FLAG_TD
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4,5.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [807]:
bal_own['Loan_restructure_code'].replace({0: 'NO_RESTR', 1: 'RESTR'}, inplace=True)

In [808]:
aggreg_restr = pd.pivot_table(bal_own, values=['Account_ID', 'BAL', 'CLT'], index=['Customer_ID'], columns=['Loan_restructure_code'], 
                             aggfunc={'Account_ID': [len], 'BAL':[sum], 'CLT':[sum]})

In [809]:
dic_restr = {'mean':'AVG', 'len':'CNT', 'sum':'SUM', 'Account_ID':'', 'Loan_restructure_code':''}

In [810]:
col_agg_restr = aggregated_index(aggreg_restr, dic_restr)
col_agg_restr = [item.strip('_') for item in col_agg_restr]
aggreg_restr.columns = col_agg_restr
aggreg_restr.reset_index(inplace=True)

In [811]:
aggreg_restr.head(10)

,Customer_ID,CNT_NO_RESTR,CNT_RESTR,BAL_SUM_NO_RESTR,BAL_SUM_RESTR,CLT_SUM_NO_RESTR,CLT_SUM_RESTR
0,11.0,1.0,NaN,502.3018,NaN,7974.0,NaN
1,12.0,NaN,1.0,NaN,0.0,NaN,1136.0
2,15.0,2.0,NaN,31430.8566,NaN,168605.0,NaN
3,16.0,1.0,NaN,1702.6200,NaN,11843.0,NaN
4,18.0,1.0,1.0,28959.6216,0.0,142147.0,1057.0
5,21.0,NaN,1.0,NaN,0.0,NaN,4945.0
6,22.0,1.0,NaN,37735.1600,NaN,155075.0,NaN
7,29.0,NaN,1.0,NaN,0.0,NaN,17162.0
8,35.0,2.0,1.0,16333.5168,0.0,72358.0,2021.0
9,41.0,1.0,NaN,0.0000,NaN,8400.0,NaN


In [812]:
aggreg_cc_status = pd.pivot_table(bal_own, values=['Account_ID', 'BAL', 'CLT'], index=['Customer_ID'], columns=['Credit_card_status'], 
                             aggfunc={'Account_ID': [len], 'BAL':[sum], 'CLT':[sum]})

In [813]:
dic_cc_status = {'mean':'AVG', 'len':'CNT', 'sum':'SUM', 'Account_ID':'', 'Credit_card_status':''}

In [814]:
col_agg_cc_status = aggregated_index(aggreg_cc_status, dic_cc_status)
col_agg_cc_status = [item.strip('_') for item in col_agg_cc_status]
col_agg_cc_status = [item.replace(' ', '_') for item in col_agg_cc_status]
col_agg_cc_status = ['CC_'+item for item in col_agg_cc_status]
aggreg_cc_status.columns = col_agg_cc_status
aggreg_cc_status.reset_index(inplace=True)

In [815]:
aggreg_cc_status.head()

,Customer_ID,CC_CNT_ACTIVE,CC_CNT_BLOCKED,CC_CNT_CANCELLED,CC_CNT_NOT_ACTIVE,CC_BAL_SUM_ACTIVE,CC_BAL_SUM_BLOCKED,CC_BAL_SUM_CANCELLED,CC_BAL_SUM_NOT_ACTIVE,CC_CLT_SUM_ACTIVE,CC_CLT_SUM_BLOCKED,CC_CLT_SUM_CANCELLED,CC_CLT_SUM_NOT_ACTIVE
0,4.0,NaN,1.0,NaN,1.0,NaN,903.87,NaN,0.0,NaN,1200.0,NaN,7800.0
1,5.0,1.0,NaN,NaN,NaN,847.80,NaN,NaN,NaN,1200.0,NaN,NaN,NaN
2,6.0,2.0,NaN,NaN,NaN,6508.52,NaN,NaN,NaN,9200.0,NaN,NaN,NaN
3,8.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,936.0,NaN,NaN,NaN,1600.0
4,14.0,1.0,1.0,NaN,NaN,2997.68,2543.64,NaN,NaN,3400.0,1400.0,NaN,NaN


In [816]:
#Activity_flag
aggreg_activity = pd.pivot_table(bal_own, values=['Account_ID', 'BAL'], index=['Customer_ID'], columns=['Activity_flag'], 
                             aggfunc={'Account_ID': [len], 'BAL': [sum]})

In [817]:
dic_activity = {'mean':'AVG', 'len':'CNT', 'sum':'SUM', 'Account_ID':'', 'Credit_card_status':''}

In [818]:
col_agg_activity = aggregated_index(aggreg_activity, dic_activity)
col_agg_activity = [item.strip('_') for item in col_agg_activity]
col_agg_activity = [item.replace(' ', '_') for item in col_agg_activity]
aggreg_activity.columns = col_agg_activity
aggreg_activity.reset_index(inplace=True)

In [819]:
aggreg_activity.head()

,Customer_ID,CNT_ACTIVE,CNT_NOT_ACTIVE,BAL_SUM_ACTIVE,BAL_SUM_NOT_ACTIVE
0,1.0,NaN,2.0,NaN,0.0
1,2.0,1.0,NaN,0.0000,NaN
2,3.0,1.0,NaN,12575.4411,NaN
3,4.0,1.0,3.0,903.8700,0.0
4,5.0,1.0,1.0,847.8000,0.0


In [820]:
#bal_own.head(20).to_excel(str(output_path + 'Bal_Own_Sample.xlsx'))

In [821]:
# replace true/false
bal_own['Debit_card'] = np.where(bal_own['Debit_card']==True,1, 0)

In [822]:
aggreg_own = pd.pivot_table(bal_own, values=['Debit_card', 'Mature_Months', 'Months_Till_Exp', 'DC_Mature_Months', 'Closed_Months'], 
                            index=['Customer_ID'], 
                             aggfunc={'Debit_card': [sum], 'Mature_Months': [max, min], 'Months_Till_Exp': [max, min],
                                     'DC_Mature_Months': [max], 'Closed_Months': [min]})

In [823]:
dic_own = {'mean':'AVG', 'len':'CNT', 'sum':'SUM', 'Account_ID':'', 'Credit_card_status':''}

In [824]:
col_agg_own = aggregated_index(aggreg_own, dic_own)
col_agg_own = [item.strip('_') for item in col_agg_own]
aggreg_own.columns = col_agg_own
aggreg_own.reset_index(inplace=True)

In [825]:
aggreg_own.head()

,Customer_ID,CLOSED_MONTHS_MIN,DC_MATURE_MONTHS_MAX,DEBIT_CARD_SUM,MATURE_MONTHS_MAX,MATURE_MONTHS_MIN,MONTHS_TILL_EXP_MAX,MONTHS_TILL_EXP_MIN
0,1.0,-1,-1,0,56,4,-1,-1
1,2.0,-1,61,1,217,217,-1,-1
2,3.0,-1,-1,0,231,231,-1,-1
3,4.0,-1,95,1,122,4,13,-1
4,5.0,-1,-1,0,173,72,0,-1


In [826]:
aggreg_bal_type = pd.pivot_table(bal_own, values=['BAL'], index=['Customer_ID'], columns=['Balance_type'], 
                             aggfunc={'BAL': [sum, len]})

In [827]:
dic_bal_type = {'mean':'AVG', 'len':'CNT', 'sum':'SUM', 'Account_ID':'', 'Credit_card_status':''}

In [828]:
col_agg_bal_type = aggregated_index(aggreg_bal_type, dic_bal_type)
col_agg_bal_type = [item.strip('_') for item in col_agg_bal_type]
aggreg_bal_type.columns = col_agg_bal_type
aggreg_bal_type.reset_index(inplace=True)

In [829]:
aggreg_bal_type.head()

,Customer_ID,BAL_CNT_CREDIT,BAL_CNT_DEBIT,BAL_SUM_CREDIT,BAL_SUM_DEBIT
0,1.0,1.0,1.0,0.00,0.0000
1,2.0,NaN,1.0,NaN,0.0000
2,3.0,NaN,1.0,NaN,12575.4411
3,4.0,3.0,1.0,903.87,0.0000
4,5.0,1.0,1.0,847.80,0.0000


In [830]:
#aggreg_bal_type #aggreg_dc
bal_agg_all = aggreg_prod.merge(aggreg_prod_flag, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 58)

In [831]:
bal_agg_all = bal_agg_all.merge(aggreg_prod_credit, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 86)

In [832]:
bal_agg_all = bal_agg_all.merge(aggreg_cat_credit, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 91)

In [833]:
bal_agg_all = bal_agg_all.merge(aggreg_cat, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 107)

In [834]:
bal_agg_all = bal_agg_all.merge(aggreg_cat_flag, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 115)

In [835]:
bal_agg_all = bal_agg_all.merge(aggreg_restr, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 121)

In [836]:
bal_agg_all = bal_agg_all.merge(aggreg_cc_status, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 133)

In [837]:
bal_agg_all = bal_agg_all.merge(aggreg_own, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 140)

In [838]:
bal_agg_all = bal_agg_all.merge(aggreg_bal_type, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 144)

In [839]:
bal_agg_all = bal_agg_all.merge(aggreg_activity, how='outer', on='Customer_ID')
bal_agg_all.shape

(100557, 148)

In [840]:
bal_agg_all.head()

,Customer_ID,CNT_AL_PROMOTION,CNT_AL_STANDARD,CNT_CA_PAYROLL,CNT_CA_STANDARD,CNT_CC_MASTERCARD_GOLD,CNT_CC_MASTERCARD_PLATINUM,CNT_CC_MASTERCARD_STANDARD,CNT_CC_VISA_ELECTRON,CNT_CC_VISA_GOLD,...,MONTHS_TILL_EXP_MAX,MONTHS_TILL_EXP_MIN,BAL_CNT_CREDIT,BAL_CNT_DEBIT,BAL_SUM_CREDIT,BAL_SUM_DEBIT,CNT_ACTIVE,CNT_NOT_ACTIVE,BAL_SUM_ACTIVE,BAL_SUM_NOT_ACTIVE
0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,-1,-1,1.0,1.0,0.00,0.0000,NaN,2.0,NaN,0.0
1,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-1,-1,NaN,1.0,NaN,0.0000,1.0,NaN,0.0000,NaN
2,3.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,-1,-1,NaN,1.0,NaN,12575.4411,1.0,NaN,12575.4411,NaN
3,4.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,...,13,-1,3.0,1.0,903.87,0.0000,1.0,3.0,903.8700,0.0
4,5.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,0,-1,1.0,1.0,847.80,0.0000,1.0,1.0,847.8000,0.0


In [841]:
del aggreg_prod, aggreg_prod_flag, aggreg_prod_credit, aggreg_cat, aggreg_cat_flag, aggreg_restr, aggreg_cc_status, 
aggreg_own, aggreg_bal_type, aggreg_activity
gc.collect()

567

Additional KPIs

In [842]:
bal_agg_all.columns

Index(['Customer_ID', 'CNT_AL_PROMOTION', 'CNT_AL_STANDARD', 'CNT_CA_PAYROLL',
       'CNT_CA_STANDARD', 'CNT_CC_MASTERCARD_GOLD',
       'CNT_CC_MASTERCARD_PLATINUM', 'CNT_CC_MASTERCARD_STANDARD',
       'CNT_CC_VISA_ELECTRON', 'CNT_CC_VISA_GOLD',
       ...
       'MONTHS_TILL_EXP_MAX', 'MONTHS_TILL_EXP_MIN', 'BAL_CNT_CREDIT',
       'BAL_CNT_DEBIT', 'BAL_SUM_CREDIT', 'BAL_SUM_DEBIT', 'CNT_ACTIVE',
       'CNT_NOT_ACTIVE', 'BAL_SUM_ACTIVE', 'BAL_SUM_NOT_ACTIVE'],
      dtype='object', length=148)

In [843]:
#bal_agg_all.head(20).to_excel(str(output_path + 'Bal_Sample.xlsx'), index=False)

In [844]:
#breakpoint

In [845]:
bal_agg_all['AT_LEAST_ONE_ACTIVE'] = np.where(bal_agg_all['CNT_ACTIVE']>0, 1, 0)

In [846]:
bal_agg_all['TOTAL_DIFF_CAT'] = bal_agg_all[['CAT_FLAG_AL', 'CAT_FLAG_CA', 'CAT_FLAG_CC', 'CAT_FLAG_MG', 'CAT_FLAG_OVD',
                                            'CAT_FLAG_PL', 'CAT_FLAG_SA', 'CAT_FLAG_TD']].sum(axis = 1, skipna = True)

In [847]:
bal_agg_all['TOTAL_PRODUCTS'] = bal_agg_all[['CAT_CNT_AL', 'CAT_CNT_CA', 'CAT_CNT_CC', 'CAT_CNT_MG', 'CAT_CNT_OVD',
                                             'CAT_CNT_PL', 'CAT_CNT_SA', 'CAT_CNT_TD']].sum(axis = 1, skipna = True)

In [848]:
bal_agg_all['TOTAL_SAVING_CNT'] = bal_agg_all[['CAT_CNT_SA', 'CAT_CNT_TD', 'CAT_CNT_CA']].sum(axis = 1, skipna = True)
bal_agg_all['TOTAL_LOANS_CNT'] = bal_agg_all[['CAT_CNT_AL', 'CAT_CNT_CC', 'CAT_CNT_MG', 'CAT_CNT_OVD', 
                                          'CAT_CNT_PL']].sum(axis = 1, skipna = True)

In [849]:
bal_agg_all['TOTAL_SAVING_BAL'] = bal_agg_all[['CAT_BAL_CA', 'CAT_BAL_SA', 'CAT_BAL_TD']].sum(axis = 1, skipna = True)

In [850]:
bal_agg_all['TOTAL_LOANS_BAL'] = bal_agg_all[['CAT_BAL_AL', 'CAT_BAL_CC', 'CAT_BAL_MG', 'CAT_BAL_OVD', 
                                                  'CAT_BAL_PL']].sum(axis = 1, skipna = True)

In [851]:
bal_agg_all['AVG_DEPOSITS_minus_LOANS'] = bal_agg_all['TOTAL_SAVING_BAL'] - bal_agg_all['TOTAL_LOANS_BAL']

In [852]:
bal_agg_all['PCT_LOANS_CNT'] = bal_agg_all['TOTAL_LOANS_CNT']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_SAVING_CNT'] = bal_agg_all['TOTAL_SAVING_CNT']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_CC_CNT'] = bal_agg_all['CAT_CNT_CC']/bal_agg_all['TOTAL_PRODUCTS']

In [853]:
#COUNTS BY CAT
bal_agg_all['PCT_CA_VS_SAVING_CNT'] = bal_agg_all['CAT_CNT_CA']/bal_agg_all['TOTAL_SAVING_CNT']
bal_agg_all['PCT_TD_VS_SAVING_CNT'] = bal_agg_all['CAT_CNT_TD']/bal_agg_all['TOTAL_SAVING_CNT']
bal_agg_all['PCT_SA_VS_SAVING_CNT'] = bal_agg_all['CAT_CNT_SA']/bal_agg_all['TOTAL_SAVING_CNT']

bal_agg_all['PCT_CC_VS_LOANS_CNT'] = bal_agg_all['CAT_CNT_CC']/bal_agg_all['TOTAL_LOANS_CNT']
bal_agg_all['PCT_AL_VS_LOANS_CNT'] = bal_agg_all['CAT_CNT_AL']/bal_agg_all['TOTAL_LOANS_CNT']
bal_agg_all['PCT_PL_VS_LOANS_CNT'] = bal_agg_all['CAT_CNT_PL']/bal_agg_all['TOTAL_LOANS_CNT']
bal_agg_all['PCT_OVD_VS_LOANS_CNT'] = bal_agg_all['CAT_CNT_OVD']/bal_agg_all['TOTAL_LOANS_CNT']
bal_agg_all['PCT_MG_VS_LOANS_CNT'] = bal_agg_all['CAT_CNT_MG']/bal_agg_all['TOTAL_LOANS_CNT']

In [854]:
#AVG BALANCES BY CAT
bal_agg_all['PCT_CA_VS_SAVING_BAL'] = bal_agg_all['CAT_BAL_CA']/bal_agg_all['TOTAL_SAVING_BAL']
bal_agg_all['PCT_TD_VS_SAVING_BAL'] = bal_agg_all['CAT_BAL_TD']/bal_agg_all['TOTAL_SAVING_BAL']
bal_agg_all['PCT_SA_VS_SAVING_BAL'] = bal_agg_all['CAT_BAL_SA']/bal_agg_all['TOTAL_SAVING_BAL']

bal_agg_all['PCT_CC_VS_LOANS_BAL'] = bal_agg_all['CAT_BAL_CC']/bal_agg_all['TOTAL_LOANS_BAL']
bal_agg_all['PCT_AL_VS_LOANS_BAL'] = bal_agg_all['CAT_BAL_AL']/bal_agg_all['TOTAL_LOANS_BAL']
bal_agg_all['PCT_PL_VS_LOANS_BAL'] = bal_agg_all['CAT_BAL_PL']/bal_agg_all['TOTAL_LOANS_BAL']
bal_agg_all['PCT_OVD_VS_LOANS_BAL'] = bal_agg_all['CAT_BAL_OVD']/bal_agg_all['TOTAL_LOANS_BAL']
bal_agg_all['PCT_MG_VS_LOANS_BAL'] = bal_agg_all['CAT_BAL_MG']/bal_agg_all['TOTAL_LOANS_BAL']

In [855]:
#CLT BY CAT
bal_agg_all['PCT_BAL_TO_LCL_AL'] = bal_agg_all['CAT_BAL_AL']/bal_agg_all['CAT_CLT_AL']
bal_agg_all['PCT_BAL_TO_LCL_CC'] = bal_agg_all['CAT_BAL_CC']/bal_agg_all['CAT_CLT_CC']
bal_agg_all['PCT_BAL_TO_LCL_MG'] = bal_agg_all['CAT_BAL_MG']/bal_agg_all['CAT_CLT_MG']
bal_agg_all['PCT_BAL_TO_LCL_OVD'] = bal_agg_all['CAT_BAL_OVD']/bal_agg_all['CAT_CLT_OVD']
bal_agg_all['PCT_BAL_TO_LCL_PL'] = bal_agg_all['CAT_BAL_PL']/bal_agg_all['CAT_CLT_PL']

In [856]:
#COUNTS BY PROD
bal_agg_all['PCT_AL_PROMOTION_CNT'] = bal_agg_all['CNT_AL_PROMOTION']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_AL_STANDARD_CNT'] = bal_agg_all['CNT_AL_STANDARD']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_CA_PAYROLL_CNT'] = bal_agg_all['CNT_CA_PAYROLL']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_CA_STANDARD_CNT'] = bal_agg_all['CNT_CA_STANDARD']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_MASTERCARD_GOLD_CNT'] = bal_agg_all['CNT_CC_MASTERCARD_GOLD']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_MASTERCARD_PLATINUM_CNT'] = bal_agg_all['CNT_CC_MASTERCARD_PLATINUM']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_MASTERCARD_STANDARD_CNT'] = bal_agg_all['CNT_CC_MASTERCARD_STANDARD']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_VISA_ELECTRON_CNT'] = bal_agg_all['CNT_CC_VISA_ELECTRON']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_VISA_GOLD_CNT'] = bal_agg_all['CNT_CC_VISA_GOLD']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_VISA_PLATINUM_CNT'] = bal_agg_all['CNT_CC_VISA_PLATINUM']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_NEW_HOUSE_CNT'] = bal_agg_all['CNT_MG_NEW_HOUSE']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_RENOVATION_CNT'] = bal_agg_all['CNT_MG_RENOVATION']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_OVD_CNT'] = bal_agg_all['CNT_OVD']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_PL_F_NON_SEC_CNT'] = bal_agg_all['CNT_PL_F_NON_SEC']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_PL_F_SEC_CNT'] = bal_agg_all['CNT_PL_F_SEC']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_PL_V_CNT'] = bal_agg_all['CNT_PL_V']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_SA_CNT'] = bal_agg_all['CNT_SA']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_TD_LT_CNT'] = bal_agg_all['CNT_TD_LT']/bal_agg_all['TOTAL_PRODUCTS']
bal_agg_all['PCT_TD_ST_CNT'] = bal_agg_all['CNT_TD_ST']/bal_agg_all['TOTAL_PRODUCTS']

In [857]:
bal_agg_all['YearMonth'] = YearMonth

In [858]:
def write_pandas_dataframe_to_blob(blob_service, df, container_name, blob_name):
    """ Write Pandas dataframe to blob storage """
    buffer = BytesIO()
    df.to_parquet(buffer)
    blob_service.create_blob_from_bytes(
        container_name=container_name, blob_name=blob_name, blob=buffer.getvalue()
    )

In [859]:
write_to_blob_parquet = 'ailiev/Balances_Master_' + YearMonth + '.parquet.gzip' #Balances_Master_2018-01-01.parquet.gzip

In [860]:
# Write to blob using pyarrow
write_pandas_dataframe_to_blob(blob_service, bal_agg_all, 'banking-dm', write_to_blob_parquet)

In [861]:
#byte_stream = BytesIO()
#try:
#   blob_service.get_blob_to_stream(container_name=container_name, blob_name='ailiev/Balances_Master_' + YearMonth + '.parquet.gzip', stream=byte_stream)
#   check = pq.read_table(source=byte_stream).to_pandas()
#   # Do work on df ...
#finally:
#   # Add finally block to ensure closure of the stream
#   byte_stream.close()

In [862]:
#check.head()

In [863]:
#bal_agg_all.head(20).to_excel(str(output_path + 'Bal_Sample.xlsx'), index=False)

In [864]:
#bal_agg_all.head()

In [865]:
#%time bal_agg_all.to_csv(str(input_path + 'Balances_Master.csv'), index=False, header=True)

In [866]:
#read in csv file
#check = pd.read_csv(str(input_path + 'Balances_Master.csv'), header=0, nrows=10)

In [867]:
#check